# Data preprocessing

In [94]:
import numpy as np
import pandas as pd
from sklearn.metrics import accuracy_score,confusion_matrix
from sklearn.tree import DecisionTreeClassifier
from sklearn.neighbors import KNeighborsClassifier
from sklearn.naive_bayes import GaussianNB, BernoulliNB
from sklearn.model_selection import KFold,cross_val_score


In [2]:
data= pd.read_csv('X:\GIt\CP 318 Worksheet Practise\Data\diabetes_data.csv')
data.head()

,pregnancies,glucose,diastolic,triceps,insulin,bmi,dpf,age,diabetes
0,6,148,72,35,0,33.6,0.627,50,1
1,1,85,66,29,0,26.6,0.351,31,0
2,8,183,64,0,0,23.3,0.672,32,1
3,1,89,66,23,94,28.1,0.167,21,0
4,0,137,40,35,168,43.1,2.288,33,1


In [3]:
data.describe()

,pregnancies,glucose,diastolic,triceps,insulin,bmi,dpf,age,diabetes
count,768.000000,768.000000,768.000000,768.000000,768.000000,768.000000,768.000000,768.000000,768.000000
mean,3.845052,120.894531,69.105469,20.536458,79.799479,31.992578,0.471876,33.240885,0.348958
std,3.369578,31.972618,19.355807,15.952218,115.244002,7.884160,0.331329,11.760232,0.476951
min,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.078000,21.000000,0.000000
25%,1.000000,99.000000,62.000000,0.000000,0.000000,27.300000,0.243750,24.000000,0.000000
50%,3.000000,117.000000,72.000000,23.000000,30.500000,32.000000,0.372500,29.000000,0.000000
75%,6.000000,140.250000,80.000000,32.000000,127.250000,36.600000,0.626250,41.000000,1.000000
max,17.000000,199.000000,122.000000,99.000000,846.000000,67.100000,2.420000,81.000000,1.000000


In [6]:
data.isnull().values.any()

False

In [24]:
y = data['diabetes']
x = data.drop(columns='diabetes')

from sklearn.model_selection import train_test_split
x_train,x_test,y_train,y_test = train_test_split(x,y,stratify = y,test_size = 0.25)

# Decision Trees

In [52]:
def predict(x_train,y_train,model):
    model.fit(x_train,y_train)
    y_hat = model.predict(x_test)
    print('Accuracy Score : ',accuracy_score(y_test,y_hat))
    print('Confusion Matrix : ',confusion_matrix(y_test,y_hat))
    return y_hat


In [106]:
pred_dt=predict(x_train,y_train,DecisionTreeClassifier())

Accuracy Score :  0.7135416666666666
Confusion Matrix :  [[92 33]
 [22 45]]


# kNN

In [157]:
pred_kNN=predict(x_test,y_test,KNeighborsClassifier(n_neighbors = 3))

Accuracy Score :  0.859375
Confusion Matrix :  [[118   7]
 [ 20  47]]


# Naive Bayes

In [109]:
#Gaussian NB
pred_gnb=predict(x_test,y_test,GaussianNB())


Accuracy Score :  0.7708333333333334
Confusion Matrix :  [[106  19]
 [ 25  42]]


In [110]:
#BernoulliNB
pred_bnb=predict(x_test,y_test,BernoulliNB())


Accuracy Score :  0.6458333333333334
Confusion Matrix :  [[117   8]
 [ 60   7]]


# Ensemble Learning

In [156]:
#Voting Classifier
from sklearn.ensemble import VotingClassifier

estimators = [#('DecisionTree',DecisionTreeClassifier())
              ('kNN',KNeighborsClassifier(n_neighbors=3))
              ,('Naive Bayes',GaussianNB())]

ensemble = VotingClassifier(estimators,voting ='hard')
ensemble.fit(x_train,y_train)
ensemble.score(x_test,y_test)


0.7760416666666666

In [144]:
#Bagging
from sklearn.ensemble import BaggingClassifier

ensemble_dt = BaggingClassifier(estimator=DecisionTreeClassifier(),n_estimators =20)
ensemble_kNN = BaggingClassifier(estimator=KNeighborsClassifier(n_neighbors = 3),n_estimators = 20)
ensemble_gNB = BaggingClassifier(estimator=GaussianNB(),n_estimators = 20,random_state=10)

def ensemble_fit(ensemble_model):
    k_folds = KFold(n_splits = 10)
    results = cross_val_score(ensemble_model,x_train,y_train,cv = k_folds).mean()
    print(f'Cross_val_score with model {ensemble_model.estimator} : ',results)


In [158]:
ensemble_fit(ensemble_dt)
ensemble_fit(ensemble_kNN)
ensemble_fit(ensemble_gNB)


Cross_val_score with model DecisionTreeClassifier() :  0.7397761645493043
Cross_val_score with model KNeighborsClassifier(n_neighbors=3) :  0.6875378100423472
Cross_val_score with model GaussianNB() :  0.7225347852389595


In [159]:
#randomforest
from sklearn.ensemble import RandomForestClassifier

ensemble_fit(RandomForestClassifier())

Cross_val_score with model DecisionTreeClassifier() :  0.7380217785843921


In [161]:
#adaboost
from sklearn.ensemble import AdaBoostClassifier

adaboost = AdaBoostClassifier(n_estimators = 20 ,random_state=1)
adaboost.fit(x_train,y_train)
adaboost.score(x_test,y_test)

0.7552083333333334

In [172]:
#gradboost
from sklearn.ensemble import GradientBoostingClassifier
model= GradientBoostingClassifier(learning_rate=0.01,random_state=1)
model.fit(x_train, y_train)
model.score(x_test, y_test)

0.7604166666666666